In [13]:
import pandas as pd
from copy import deepcopy
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
import warnings
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import time

# Игнорирование FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [14]:
df = pd.read_csv('./Titanic-Dataset.csv')

features = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy()
target_class = df['Survived'].values  # для задачи классификации
target_reg = df['Fare'].values       # для задачи регрессии

features['Sex'] = features['Sex'].map({'male': 0, 'female': 1})
features['Embarked'] = features['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

features['Age'].fillna(features['Age'].mean(), inplace=True)
features['Embarked'].fillna(features['Embarked'].mode()[0], inplace=True)

X = features.values

X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(
    X, target_class, test_size=0.2, random_state=42
)

X_train_r, X_val_r, y_train_r, y_val_r = train_test_split(
    X, target_reg, test_size=0.2, random_state=42
)

In [15]:
from Classifier import TreeClassifier
from Regressor import TreeRegressor

tree_ent = TreeClassifier(criterion='entropy', max_depth=10, min_samples_split=2)
t1 = time.time()
tree_ent.fit(X_train_c, y_train_c)
t2 = time.time()
print(f"Энтропия (время обучения): {t2 - t1:.4f}s")

# Предсказание
t1 = time.time()
y_pred_ent = tree_ent.predict(X_val_c)
t2 = time.time()
acc_ent = accuracy_score(y_val_c, y_pred_ent)
f1_ent = f1_score(y_val_c, y_pred_ent)
print(f"Энтропия (время предсказания): {t2 - t1:.4f}s")
print("Энтропия: Accuracy =", acc_ent, "F1 =", f1_ent)

t1 = time.time()
tree_ent.prune(X_val_c, y_val_c)
t2 = time.time()
print(f"Энтропия (время обрезки): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_ent_pruned = tree_ent.predict(X_val_c)
t2 = time.time()
acc_ent_pruned = accuracy_score(y_val_c, y_pred_ent_pruned)
f1_ent_pruned = f1_score(y_val_c, y_pred_ent_pruned)
print(f"Энтропия (время предсказания после обрезки): {t2 - t1:.4f}s")
print("Энтропия (после обрезки): Accuracy =", acc_ent_pruned, "F1 =", f1_ent_pruned)

print()

tree_donskoy = TreeClassifier(criterion='donskoy', max_depth=10, min_samples_split=2)
t1 = time.time()
tree_donskoy.fit(X_train_c, y_train_c)
t2 = time.time()
print(f"Критерий Донского (время обучения): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_donskoy = tree_donskoy.predict(X_val_c)
t2 = time.time()
acc_donskoy = accuracy_score(y_val_c, y_pred_donskoy)
f1_donskoy = f1_score(y_val_c, y_pred_donskoy)
print(f"Критерий Донского (время предсказания): {t2 - t1:.4f}s")
print("Критерий Донского: Accuracy =", acc_donskoy, "F1 =", f1_donskoy)

t1 = time.time()
tree_donskoy.prune(X_val_c, y_val_c)
t2 = time.time()
print(f"Критерий Донского (время обрезки): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_donskoy_pruned = tree_donskoy.predict(X_val_c)
t2 = time.time()
acc_ent_pruned = accuracy_score(y_val_c, y_pred_donskoy_pruned)
f1_ent_pruned = f1_score(y_val_c, y_pred_donskoy_pruned)
print(f"Критерий Донского (время предсказания после обрезки): {t2 - t1:.4f}s")
print("Критерий Донского (после обрезки): Accuracy =", acc_ent_pruned, "F1 =", f1_ent_pruned)

Энтропия (время обучения): 0.7230s
Энтропия (время предсказания): 0.0050s
Энтропия: Accuracy = 0.7932960893854749 F1 = 0.7299270072992701
Энтропия (время обрезки): 0.0490s
Энтропия (время предсказания после обрезки): 0.0040s
Энтропия (после обрезки): Accuracy = 0.7932960893854749 F1 = 0.7299270072992701

Критерий Донского (время обучения): 0.5740s
Критерий Донского (время предсказания): 0.0050s
Критерий Донского: Accuracy = 0.7932960893854749 F1 = 0.7299270072992701
Критерий Донского (время обрезки): 0.0480s
Критерий Донского (время предсказания после обрезки): 0.0050s
Критерий Донского (после обрезки): Accuracy = 0.7932960893854749 F1 = 0.7299270072992701


In [16]:
tree_reg = TreeRegressor(min_samples_split=5)

t1 = time.time()
tree_reg.fit(X_train_r, y_train_r)
t2 = time.time()
print(f"Регрессия (время обучения): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_reg = tree_reg.predict(X_val_r)
t2 = time.time()
mse_reg = mean_squared_error(y_val_r, y_pred_reg)
r2_reg = r2_score(y_val_r, y_pred_reg)
print(f"Регрессия (время предсказания): {t2 - t1:.4f}s")
print("Регрессия: MSE =", mse_reg, "R^2 =", r2_reg)

t1 = time.time()
tree_reg.prune(X_val_r, y_val_r)
t2 = time.time()
print(f"Регрессия (время обрезки): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_reg_pruned = tree_reg.predict(X_val_r)
t2 = time.time()
mse_reg_pruned = mean_squared_error(y_val_r, y_pred_reg_pruned)
r2_reg = r2_score(y_val_r, y_pred_reg_pruned)
print(f"Регрессия (время предсказания после обрезки): {t2 - t1:.4f}s")
print("Регрессия (после обрезки):MSE =", mse_reg, "R^2 =", r2_reg)

Регрессия (время обучения): 0.4570s
Регрессия (время предсказания): 0.0030s
Регрессия: MSE = 0.36631807725675036 R^2 = 0.999763273190162
Регрессия (время обрезки): 0.2040s
Регрессия (время предсказания после обрезки): 0.0040s
Регрессия (после обрезки):MSE = 0.36631807725675036 R^2 = 0.999763273190162


In [17]:
clf_sklearn = DecisionTreeClassifier(
    criterion='entropy', max_depth=10, min_samples_split=2, random_state=42
)
t1 = time.time()
clf_sklearn.fit(X_train_c, y_train_c)
t2 = time.time()
print(f"sklearn Энтропия (время обучения): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_sklearn = clf_sklearn.predict(X_val_c)
t2 = time.time()
acc_sklearn = accuracy_score(y_val_c, y_pred_sklearn)
print(f"sklearn Энтропия (время предсказания): {t2 - t1:.4f}s")
print("sklearn Энтропия Accuracy =", acc_sklearn, "F1 =", f1_score(y_val_c, y_pred_sklearn))

sklearn Энтропия (время обучения): 0.0030s
sklearn Энтропия (время предсказания): 0.0000s
sklearn Энтропия Accuracy = 0.8044692737430168 F1 = 0.7445255474452555


In [18]:
reg = DecisionTreeRegressor(
    criterion='squared_error', max_depth=10, min_samples_split=5, random_state=42
)
t1 = time.time()
reg.fit(X_train_r, y_train_r)
t2 = time.time()
print(f"sklearn Регрессия (время обучения): {t2 - t1:.4f}s")

t1 = time.time()
y_pred_sklearn_reg = reg.predict(X_val_r)
t2 = time.time()
mse_sklearn = mean_squared_error(y_val_r, y_pred_sklearn_reg)
r2_sklearn = r2_score(y_val_r, y_pred_sklearn_reg)
print(f"sklearn Регрессия (время предсказание): {t2 - t1:.4f}s")
print("sklearn Регрессия: MSE =", mse_sklearn, "R^2 =", r2_sklearn)

sklearn Регрессия (время обучения): 0.0020s
sklearn Регрессия (время предсказание): 0.0010s
sklearn Регрессия: MSE = 0.28196101211523195 R^2 = 0.9998177875047921
